In [ ]:
%pip install semantic-link==0.11.1 semantic-link-labs==0.11.2  > /dev/null 2>&1
import pandas as pd
import requests 
import sempy_labs as labs
import sempy.fabric as semfabric

StatementMeta(, 07d80e2d-a99f-4210-9687-6821d94cc581, 8, Finished, Available, Finished)

Note: you may need to restart the kernel to use updated packages.



In [ ]:
## You can change these as you like
lakehouse_name = 'Baraa_LH'
warehouse_name = 'Baraa_WH'
gold_notebook_name = 'gold_layer_processing_notebook'
silver_notebook_name = 'silver_layer_processing_notebook'
timezone = "Africa/Lagos"

## Don't Change these ⚠️⚠️⚠️⚠️⚠️⚠️⚠️
crm_data_relative_path = "datasets/source_crm"
erp_data_relative_path = "datasets/source_erp"

silver_notebook_url = 'https://raw.githubusercontent.com/maleek004/auto-fabric-setup-baraa/refs/heads/main/Create_silver_layer_Notebook.ipynb'
gold_notebook_url= 'https://raw.githubusercontent.com/maleek004/auto-fabric-setup-baraa/refs/heads/main/Load_gold_layer.ipynb'

crm_dataset_urls = {"cust_info.csv":"https://raw.githubusercontent.com/DataWithBaraa/sql-data-warehouse-project/refs/heads/main/datasets/source_crm/cust_info.csv"
                    ,"prd_info.csv":"https://raw.githubusercontent.com/DataWithBaraa/sql-data-warehouse-project/refs/heads/main/datasets/source_crm/prd_info.csv"
                    ,"sales_details.csv":"https://raw.githubusercontent.com/DataWithBaraa/sql-data-warehouse-project/refs/heads/main/datasets/source_crm/sales_details.csv"}

erp_dataset_urls = {"CUST_AZ12.csv":"https://raw.githubusercontent.com/DataWithBaraa/sql-data-warehouse-project/refs/heads/main/datasets/source_erp/CUST_AZ12.csv"
                    ,"LOC_A101.csv":"https://raw.githubusercontent.com/DataWithBaraa/sql-data-warehouse-project/refs/heads/main/datasets/source_erp/LOC_A101.csv"
                    ,"PX_CAT_G1V2.csv":"https://raw.githubusercontent.com/DataWithBaraa/sql-data-warehouse-project/refs/heads/main/datasets/source_erp/PX_CAT_G1V2.csv"}

StatementMeta(, 07d80e2d-a99f-4210-9687-6821d94cc581, 10, Finished, Available, Finished)

In [4]:
def fabriclient_post(url, request_body):

    client = semfabric.FabricRestClient()
    
    response = client.request(method = "POST", path_or_url=url, lro_wait=True, json = request_body)
    print(response.status_code)
    print(response.text)
    response.raise_for_status()
    
    response_json = response.json()
    return response_json.get('id')

StatementMeta(, 07d80e2d-a99f-4210-9687-6821d94cc581, 12, Finished, Available, Finished)

In [ ]:
# creating a new lakehouse where our data will be downladed will be downloaded into and will alsoe be used as the siver layer

lakehouse = notebookutils.lakehouse.create(lakehouse_name)    
lakehouse_id = lakehouse['id']
workspace_id = lakehouse['workspaceId']
abfs_path = lakehouse.get('properties',{}).get('abfsPath')
abfs_path

StatementMeta(, 07d80e2d-a99f-4210-9687-6821d94cc581, 13, Finished, Available, Finished)

'abfss://63bd4ac8-18b7-4ec1-8ee9-681feee6113e@onelake.dfs.fabric.microsoft.com/84285eaa-9ab8-4dc9-8ecc-005a40bcda83'

In [ ]:
# Creating a warehouse where our gold layer views and tables will be stored
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/warehouses"
request_body = {"displayName": warehouse_name}

warehouse_id = fabriclient_post(url,request_body)
warehouse_id

In [ ]:
# creating directories in our destination lakehouse (that this notebook is not attached to, hence the use of abfs_path) 
notebookutils.fs.mkdirs(f'{abfs_path}/Files/{crm_data_relative_path}')
notebookutils.fs.mkdirs(f'{abfs_path}/Files/{erp_data_relative_path}')

In [ ]:
##  mount the Files section of the new lakehouse 
files_directory = abfs_path + '/Files'
mount_point = "/mnt/lakehouse/" + lakehouse_name + "/Files"
notebookutils.fs.mount(files_directory, mount_point)
base_dir_local_path = notebookutils.fs.getMountPath(mount_point)
base_dir_local_path

In [ ]:
crm_data_full_local_path = f'{base_dir_local_path}/{crm_data_relative_path}'
erp_data_full_local_path = f'{base_dir_local_path}/{erp_data_relative_path}'


In [ ]:

def download_datasets(dataset_urls, save_dir="."):
    for filename, url in dataset_urls.items():
        print(f"Downloading {filename}...")
        try:
            response = requests.get(url, timeout=15)
            response.raise_for_status()  # Raise exception for HTTP errors
            with open(f"{save_dir}/{filename}", "wb") as f:
                f.write(response.content)
            print(f"✅ {filename} downloaded successfully.")
        except requests.exceptions.RequestException as e:
            print(f"❌ Failed to download {filename}: {e}")


download_datasets(crm_dataset_urls, crm_data_full_local_path)
download_datasets(erp_dataset_urls, erp_data_full_local_path)


StatementMeta(, 07d80e2d-a99f-4210-9687-6821d94cc581, 20, Finished, Available, Finished)

✅ cust_info.csv downloaded successfully.
✅ prd_info.csv downloaded successfully.
✅ sales_details.csv downloaded successfully.
✅ CUST_AZ12.csv downloaded successfully.
✅ LOC_A101.csv downloaded successfully.
✅ PX_CAT_G1V2.csv downloaded successfully.


In [ ]:


def import_notebook(notebook_import_name, githuburl, update_lakehouse=False , workspace_id=None, lakehouse_name=None ) -> str:
    # import notebook and return notebook_id
    result = labs.import_notebook_from_web( notebook_name=notebook_import_name, url=githuburl)
        
    # update the default lakehouse (only if allowed)
    if update_lakehouse:
        if not workspace_id or not lakehouse_name:
            raise ValueError("workspace_id and lakehouse_name must be provided when update_lakehouse=True")
        notebookutils.notebook.updateDefinition(
            name=notebook_import_name, 
            workspaceId=workspace_id, 
            defaultLakehouse=lakehouse_name, 
            defaultLakehouseWorkspace=workspace_id
        )
    
    # Return notebook id
    notebook_id = semfabric.resolve_item_id(item_name=notebook_import_name, type="Notebook")

    print(f"notebookname: {notebook_import_name}, notebook_id: {notebook_id}")

    return notebook_id


#import notebooks and get Notebook Ids for all 3 notebooks to be used in subsequent steps
silver_layer_notebook_id = import_notebook(notebook_import_name=silver_notebook_name, githuburl=silver_notebook_url, update_lakehouse=True, workspace_id=workspace_id, lakehouse_name = lakehouse_name)
gold_layer_notebook_id = import_notebook(gold_notebook_name,gold_notebook_url)

StatementMeta(, 07d80e2d-a99f-4210-9687-6821d94cc581, 21, Finished, Available, Finished)

🟢 The 'silver_layer_processing_notebook' notebook has been successfully created within the 'Baraa e2e test2' workspace.
notebookname: silver_layer_processing_notebook, notebook_id: 92b2bbcf-6433-42bb-96aa-7d9284f41fab
🟢 The 'gold_layer_processing_notebook' notebook has been successfully created within the 'Baraa e2e test2' workspace.
notebookname: gold_layer_processing_notebook, notebook_id: 41614e4d-1858-409e-b1a3-8d57dcd0723d


In [ ]:
DAG = {
    "activities": [
        {
            "name": silver_notebook_name, 
            "path": silver_notebook_name, 
            "timeoutPerCellInSeconds": 240,
            "args": {"useRootDefaultLakehouse": True, "abfs_path":abfs_path, "workspaceId": workspace_id , "lakehouseId": lakehouse_id , "timezone" : timezone}
        },
        {
            "name": gold_notebook_name,
            "path": gold_notebook_name,
            "timeoutPerCellInSeconds": 240,
            "args":{"lakehouse_name": lakehouse_name , "warehouse_name":warehouse_name},
            "dependencies": [silver_notebook_name]
        }
    ],
    "timeoutInSeconds": 480, # max 4 mins for the entire pipeline
}

notebookutils.notebook.runMultiple(DAG, {"displayDAGViaGraphviz":True})